<a href="https://colab.research.google.com/github/Alfred78w/AI_project/blob/main/RAG_and_Bio_and_scraper_graph_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-mistralai

In [ ]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.1 MB/s eta 0:00:00


# Load

In [ ]:
doc_path = (
    "/content/doc.pdf"
)

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(doc_path)
documents = loader.load()
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

# Split

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)
# Convert the list of Documents to a single string
text = "".join([doc.page_content for doc in documents])
# Now split the text
texts = text_splitter.split_text(text)

In [ ]:
texts

["bonjour je m'appelle sophie je suis infirmière depuis 2016 et j'interviens domicile depuis quatre",
 'depuis quatre ans je suis également diététicienne depuis 2004 sur le secteur donc du bas rhin et',
 "donc du bas rhin et de strasbourg plus précisément donc en milieu urbain aujourd'hui nous allons",
 "nous allons parler d'éducation thérapeutique du patient diabétique et pour commencer je pense",
 "commencer je pense qu'une petite définition ne fait jamais  de mal donc je vais vous lire à celle",
 "vous lire à celle de l'om est ce qui a été publié en 96 donc l'éducation thérapeutique du patient",
 'du patient vise à aider les patients à acquérir au maintenir les compétences dont ils ont besoin',
 'dont ils ont besoin pour gérer au mieux leur vie avec une maladie chronique donc en réalité',
 'donc en réalité concrètement ça veut dire que le professionnel de santé doit aider son patient à',
 'aider son patient à le rendre acteur de  ses soins le patient ou son entourage par rapport aux

# Embed + store

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 39.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

In [ ]:
# Convert the list of strings to a list of Documents
documents = [Document(page_content=t) for t in texts]

# Now create the FAISS vector store
# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector = FAISS.from_documents(documents, embedding_model)

In [ ]:
vector

In [ ]:
documents

[Document(metadata={}, page_content="bonjour je m'appelle sophie je suis infirmière depuis 2016 et j'interviens domicile depuis quatre"),
 Document(metadata={}, page_content='depuis quatre ans je suis également diététicienne depuis 2004 sur le secteur donc du bas rhin et'),
 Document(metadata={}, page_content="donc du bas rhin et de strasbourg plus précisément donc en milieu urbain aujourd'hui nous allons"),
 Document(metadata={}, page_content="nous allons parler d'éducation thérapeutique du patient diabétique et pour commencer je pense"),
 Document(metadata={}, page_content="commencer je pense qu'une petite définition ne fait jamais  de mal donc je vais vous lire à celle"),
 Document(metadata={}, page_content="vous lire à celle de l'om est ce qui a été publié en 96 donc l'éducation thérapeutique du patient"),
 Document(metadata={}, page_content='du patient vise à aider les patients à acquérir au maintenir les compétences dont ils ont besoin'),
 Document(metadata={}, page_content='dont

Local save

In [ ]:
vector.save_local("faiss_index")

#Retrieve

In [ ]:
question = "Comment prevenir le diabète ?"
retrieved_docs = vector.similarity_search(question)


In [ ]:
retrieved_docs

[Document(metadata={}, page_content="point de vue de l'alimentation puisque  une alimentation sainealimentation saine équilibrée variée  est nécessaire pour réguler au mieux la  glycémie et doncglycémie et donc pour éviter toutes les  complications liées au diabète  la première approche àpremière approche à avoir avant toute  chose c'est d'abord de savoir se  remettre en question d'unen question d'un point de vue  professionnel puisque on ne prend plus  en charge aujourd'hui leaujourd'hui le diabète à  domicile où le secteur hospitalier"),
 Document(metadata={}, page_content="qu'il s'agisse de  l'infirmière libéralelibérale qui est la plus à  même de connaître son patient  puisqu'elle intervient au quotidienau quotidien  qu'il s'agisse de son médecin  généraliste puisqu'il va être son  référent médicalréférent médical le plus régulier mais  également de son diabétologue  idéalement il estidéalement il est important pour moi à  mon sens d'un point de vue diététique  justement que lejust

#Answers questions based on contexte

choix du model

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

relier avec Rag

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
reader_model_name =  "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(reader_model_name)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
contejxt for= retrieved_docs

final_prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": """
                                        Act theurapy coach to answer the question.
                                        If the answer cannot be deduced from the context,  ask for externals sources.
                                        Answer must be in the same language as provided question.
                                        Answer must be a friendly and funny.

                                        """},  # You can fill this with appropriate system messages
        {"role": "user", "content": f"Context:\n{context}\n---\ Here is the question :  {question}"}
    ],
    tokenize=False,
    add_generation_prompt=True
        )

print("=> Generating answer...")

answer = model.generate_content(final_prompt)

NameError: name 'retrieved_docs' is not defined

In [ ]:
answer.text

'On dirait que vous voulez en savoir plus sur la prévention du diabète. Je ne suis pas médecin, mais je peux vous aider à trouver des informations en ligne sur ce sujet. Voici quelques ressources qui pourraient vous être utiles\xa0:\n\n* [Comment prévenir le diabète](https://www.passeportsante.net/fr/Maux/Problemes/Fiche.aspx?doc=diabete-type-2-prevention)\n* [Prévenir le diabète de type 2 - Diabète Québec](https://diabete.qc.ca/fr/vivre-avec-le-diabete/comprendre-le-diabete/quest-ce-que-le-diabete/diabete-de-type-2/prevention)\n* [Prévenir le diabète de type 2 - Fédération Française des Diabétiques](https://www.federationdesdiabetiques.org/diabete-type-2/tout-savoir/comprendre-le-diabete-de-type-2/prevention)\n\nSi vous avez toujours des questions après avoir consulté ces ressources, je vous recommande de parler à votre médecin ou à un autre professionnel de la santé.'

#Scraper



*   scrapegraphai



In [ ]:
!pip install scrapegraphai

!playwright install

Removing unused browser at /root/.cache/ms-playwright/chromium-1148
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 45.7s161.3 MiB [] 0% 89.9s161.3 MiB [] 0% 13.4s161.3 MiB [] 0% 9.3s161.3 MiB [] 1% 7.1s161.3 MiB [] 1% 6.0s161.3 MiB [] 2% 5.3s161.3 MiB [] 2% 4.8s161.3 MiB [] 3% 4.9s161.3 MiB [] 4% 4.6s161.3 MiB [] 4% 4.5s161.3 MiB [] 4% 4.4s161.3 MiB [] 5% 4.2s161.3 MiB [] 5% 4.4s161.3 MiB [] 6% 4.2s161.3 MiB [] 6% 4.7s161.3 MiB [] 6% 4.9s161.3 MiB [] 6% 5.0s161.3 MiB [] 7% 5.0s161.3 MiB [] 7% 4.9s161.3 MiB [] 8% 4.9s161.3 MiB [] 8% 4.7s161.3 MiB [] 9% 4.5s161.3 MiB [] 9% 4.7s161.3 MiB [] 10% 4.4s161.3 MiB [] 10% 4.5s161.3 MiB [] 11% 4.4s161.3 MiB [] 11% 4.3s161.3 MiB [] 12% 4.2s161.3 MiB [] 13% 4.2s161.3 MiB [] 13% 4.0s161.3 MiB [] 14% 3.9s161.3 MiB [] 14% 4.0s161.3 MiB [] 15% 4.0s161.3 MiB [] 15% 3.9s161.3 MiB [] 16% 3.9s161.3 MiB [] 16% 3.8s161.3 MiB [] 18% 3.6s161.3 MiB [] 19% 3.4s161.3 MiB [] 20% 3.2s161.3 MiB [] 21% 3.0s161.3 MiB [] 22% 2.9s161.3 MiB [] 23% 2.9s161.3 MiB [] 24% 2.7s161.3 MiB

In [ ]:
!pip install nest_asyncio
!pip install -q -U google-generativeai

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

reader_model_name =  "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(reader_model_name)

# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

1. source="https://www.federationdesdiabetiques.org/diabete/glycemie"

2. https://www.ajd-diabete.fr/
3. https://www.sfdiabete.org/ (pas encore fini)
4. https://www.inserm.fr/
5. https://naitreetgrandir.com/fr/sante/az/
6. https://www.passeportsante.net/
7. https://www.diabete.fr/
8. https://www.ffrdiabete.org/
9. https://www.dinnosante.fr/







In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 23.6 MB/s eta 0:00:00


In [ ]:
https://www.diabete.qc.ca/le-diabete/informations-sur-le-diabete/quest-ce-que-le-diabete/
https://diabetesatlas.org

https://iris.who.int/

https://www.dinnosante.fr/comprendre-le-diabete/diabete

www.sante.gov.ma

https://www.diabete.qc.ca/ressources/documentation/

In [ ]:
from re import U
import json
from scrapegraphai.graphs import SmartScraperGraph, SmartScraperMultiGraph
from google.colab import userdata


import nest_asyncio
nest_asyncio.apply()
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# ... (your existing code) ...


# Define the configuration for the scraping pipeline
# graph_config = {
#     "llm": {
#         "api_key": "YOUR_OPENAI_APIKEY",
#         "model": "openai/gpt-4o-mini",
#     },
#     "verbose": True,
#     "headless": False,
# }

graph_config = {
    "llm": {
        "api_key": GOOGLE_API_KEY,
        "model": "gemini-pro"
    },
}

# Create the SmartScraperGraph instance
smart_scraper_graph = SmartScraperGraph(
    prompt="""Retrieve the all internal link from the web site.
              """,

    source="https://www.sante.gov.ma",
    config=graph_config
)

# Run the pipeline
# result = smart_scraper_graph.run()
# print(json.dumps(result, indent=4))


result = smart_scraper_graph.run()
# print(json.dumps(result, indent=4))

Found providers ['google_genai'] for model gemini-pro, using google_genai.
If it was not intended please specify the model provider in the graph configuration


ERROR:web-loader:Attempt 1 failed: 
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 526, in wrap_api_call
    return await cb()
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 89, in _inner_send
    callback = self._connection._send_message_to_server(
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_connection.py", line 325, in _send_message_to_server
    raise self._closed_error
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/

In [ ]:
len(result["internal_links"])

32

In [ ]:
result

{'internal_links': ['/prestation-personnalisee',
  '/suivi-de-votre-enfant',
  '/outils-digitaux',
  '/gestion-diabete',
  '/comprendre-le-diabete',
  '/dispositifs-medicaux',
  '/qui-sommes-nous',
  '/nous-rejoindre',
  '/node/26',
  '/comprendre-le-diabete/hyperglycemie',
  '/comprendre-le-diabete/insuline',
  '/comprendre-le-diabete/diabete-type-1',
  '/comprendre-le-diabete/diabete-type-2',
  '/comprendre-le-diabete/diabete-gestationnel',
  '/comprendre-le-diabete/retinopathie-diabetique',
  '/comprendre-le-diabete/neuropathie-diabetique',
  '/comprendre-le-diabete/pied-diabetique',
  '/comprendre-le-diabete/diabetoporose',
  '/comprendre-le-diabete/la-nephropathie-diabetique',
  '/node/463',
  '/dispositifs-medicaux/glucometre',
  '/dispositifs-medicaux/mcg',
  '/comprendre-le-diabete/insulinotherapie',
  '/dispositifs-medicaux/pompe-a-insuline',
  '/prise-contact',
  '/comprendre-le-diabete/diabete-chez-l-enfant',
  '/qui-sommes-nous',
  '/contact',
  '/nous-rejoindre',
  '/menti

In [ ]:
# context = retrieved_docs
model = genai.GenerativeModel('gemini-pro')

question = "Filters this links."

final_prompt = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": """
                                        Retrieve useful link.
                                        Don't retrieve the link that give informations simmialr to : my account, home, cart, about us, our actions.
                                        Don't retrieve the link that redirect to another web site or social network.

                                        """},  # You can fill this with appropriate system messages
        {"role": "user", "content": f"Links:\n{result}\n---\ Here is the question :  {question}"}
    ],
    tokenize=False,
    add_generation_prompt=True
        )

print("=> Generating answer...")

answer = model.generate_content(final_prompt)

=> Generating answer...


In [ ]:
answer.text

"Filtered links:\n{'internal_links': ['https://www.diabete.qc.ca/service/programme/', 'https://www.diabete.qc.ca/service/service-infodiabete-2/', 'https://www.diabete.qc.ca/notre-mission-notre-vision-nos-valeurs/', 'https://www.diabete.qc.ca/conseil-dadministration/', 'https://www.diabete.qc.ca/conseil-professionnel/', 'https://www.diabete.qc.ca/a-propos-de-diabete-quebec/notre-rapport-dimpact/', 'https://www.diabete.qc.ca/nous-joindre/', 'https://www.diabete.qc.ca/actualites/', 'https://www.diabete.qc.ca/le-diabete-en-questions/', 'https://www.diabete.qc.ca/le-diabete-en-chiffres-2/', 'https://www.diabete.qc.ca/pourquoi-devenir-membre/', 'https://www.diabete.qc.ca/renouvellement-dadhesion/', 'https://www.diabete.qc.ca/changement-dadresse/', 'https://www.diabete.qc.ca/en/diabetes/information-on-diabetes/what-is-diabetes/', 'https://www.diabete.qc.ca/faq/', 'https://www.diabete.qc.ca/le-diabete/informations-sur-le-diabete/quest-ce-que-le-diabete/', 'https://www.diabete.qc.ca/le-diabete/

In [ ]:
import ast

# Convertir la chaîne en dictionnaire Python
data_dict = ast.literal_eval(answer.text)
print(data_dict)

# Nettoyer les liens pour supprimer les caractères d'échappement
internal_links = [link.replace("\\n", "") for link in data_dict['internal_links']]

# Afficher la liste nettoyée
# for link in internal_links:
#     print(link)

# data_dict['internal_links']
internal_links

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
len(internal_links)

82

In [ ]:
!pip install requests beautifulsoup4 scrapy selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

def scrape_site_to_json(url, output_file):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        page_title = soup.title.string.strip() if soup.title else "Sans titre"

        body_content = soup.body
        if body_content:
            for header in body_content.find_all(['header', 'footer']):
                header.decompose()
            page_content = body_content.get_text(separator='\n').strip()
        else:
            page_content = ""

        try:
            with open(output_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
        except (FileNotFoundError, json.JSONDecodeError):
            data = {}

        data[page_title] = page_content

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)

        print(f"Les données de la page '{page_title}' ont été ajoutées à {output_file}.")
    except requests.RequestException as e:
        print(f"Erreur lors de la récupération du site: {e}")
    except Exception as e:
        print(f"Erreur inattendue: {e}")

if __name__ == "__main__":

    urls = internal_links
    # url = input("Entrez l'URL du site web à récupérer: ").strip()
    for url in urls:
        output_file = "collecte_pages.json"
        scrape_site_to_json("https://www.passeportsante.net"+url, output_file)
        time.sleep(5)



Les données de la page 'Problèmes et maladies - PasseportSanté' ont été ajoutées à collecte_pages.json.
Les données de la page 'Maladies - PasseportSanté.net' ont été ajoutées à collecte_pages.json.
Les données de la page 'Symptômes - PasseportSanté.net' ont été ajoutées à collecte_pages.json.
Les données de la page 'Parties du corps : encyclopédie complète' ont été ajoutées à collecte_pages.json.
Les données de la page 'Analyses médicales - PasseportSanté' ont été ajoutées à collecte_pages.json.
Les données de la page 'Spécialités médicales - quel spécialiste de santé pour moi ?' ont été ajoutées à collecte_pages.json.
Les données de la page 'Examens et opérations - PasseportSanté.net' ont été ajoutées à collecte_pages.json.
Les données de la page 'Matériel médical : à quoi çà sert, comment çà s'utilise ?' ont été ajoutées à collecte_pages.json.
Les données de la page 'Santé de la femme : nos conseils contraception, hormones, menstruations' ont été ajoutées à collecte_pages.json.
Les 

In [ ]:
!pip install requests beautifulsoup4 scrapy selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_all_links(base_url):
    visited = set()
    to_visit = [base_url]
    all_texts = []

    while to_visit:
        current_url = to_visit.pop(0)
        if current_url in visited:
            continue

        visited.add(current_url)
        try:
            response = requests.get(current_url, timeout=10)
            soup = BeautifulSoup(response.text, 'html.parser')

            # Récupérer le texte de la page
            page_text = soup.get_text(strip=True)
            all_texts.append(page_text)

            # Trouver tous les liens internes
            for a_tag in soup.find_all('a', href=True):
                link = urljoin(base_url, a_tag['href'])
                if base_url in link and link not in visited:
                    to_visit.append(link)
        except Exception as e:
            print(f"Erreur lors de la récupération de {current_url}: {e}")

    return all_texts

base_url = "https://www.federationdesdiabetiques.org/diabete/glycemie"
all_texts = get_all_links(base_url)

# Enregistrer le texte dans un fichier
with open("site_texts.txt", "w", encoding="utf-8") as f:
    f.write("\n\n".join(all_texts))


#Decoupage fichier

In [ ]:
import json

def couper_fichier_json(nom_fichier_source, fichier_sortie_1, fichier_sortie_2):
    # Lire toutes les lignes du fichier source
    with open(nom_fichier_source, 'r', encoding='utf-8') as fichier_source:
        lignes = fichier_source.readlines()

    # Calculer l'index pour diviser les lignes en deux
    milieu = len(lignes) // 2

    # Écrire la première moitié dans le premier fichier
    with open(fichier_sortie_1, 'w', encoding='utf-8') as fichier1:
        fichier1.writelines(lignes[:milieu])

    # Écrire la seconde moitié dans le second fichier
    with open(fichier_sortie_2, 'w', encoding='utf-8') as fichier2:
        fichier2.writelines(lignes[milieu:])

    print(f"Le fichier {nom_fichier_source} a été divisé en deux fichiers :")
    print(f"- Première partie : {fichier_sortie_1}")
    print(f"- Deuxième partie : {fichier_sortie_2}")


# Appel de la fonction avec des exemples de fichiers
couper_fichier_json("partie1a_1.json", "partie1a_1_1.json", "parie1a_1_2.json")


Le fichier partie1a_1.json a été divisé en deux fichiers :
- Première partie : partie1a_1_1.json
- Deuxième partie : parie1a_1_2.json


In [ ]:
import os
import json

def couper_fichier_json(nom_fichier_source, dossier_sortie, n):
    """
    Divise un fichier JSON en `n` parties et enregistre chaque partie dans un dossier spécifié.

    :param nom_fichier_source: Chemin du fichier JSON source.
    :param dossier_sortie: Chemin du dossier où les fichiers découpés seront sauvegardés.
    :param n: Nombre total de fichiers à générer.
    """
    # Lire toutes les lignes du fichier source
    with open(nom_fichier_source, 'r', encoding='utf-8') as fichier_source:
        lignes = fichier_source.readlines()

    # Calculer le nombre de lignes par fichier
    total_lignes = len(lignes)
    lignes_par_fichier = total_lignes // n + (1 if total_lignes % n != 0 else 0)

    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(dossier_sortie):
        os.makedirs(dossier_sortie)

    # Découper et écrire dans les fichiers
    for i in range(n):
        debut = i * lignes_par_fichier
        fin = min(debut + lignes_par_fichier, total_lignes)
        fichier_sortie = os.path.join(dossier_sortie, f"partie_{i + 1}.json")

        # Écrire la partie dans un fichier
        with open(fichier_sortie, 'w', encoding='utf-8') as fichier_partie:
            fichier_partie.writelines(lignes[debut:fin])

        print(f"Partie {i + 1} sauvegardée sous {fichier_sortie}")

# Exemple d'utilisation
# Nom du fichier source et du dossier de sortie
nom_fichier_source = "train.json"
dossier_sortie = "dossier_decoupes"
n = 300  # Nombre total de fichiers à générer

couper_fichier_json(nom_fichier_source, dossier_sortie, n)


Partie 1 sauvegardée sous dossier_decoupes/partie_1.json
Partie 2 sauvegardée sous dossier_decoupes/partie_2.json
Partie 3 sauvegardée sous dossier_decoupes/partie_3.json
Partie 4 sauvegardée sous dossier_decoupes/partie_4.json
Partie 5 sauvegardée sous dossier_decoupes/partie_5.json
Partie 6 sauvegardée sous dossier_decoupes/partie_6.json
Partie 7 sauvegardée sous dossier_decoupes/partie_7.json
Partie 8 sauvegardée sous dossier_decoupes/partie_8.json
Partie 9 sauvegardée sous dossier_decoupes/partie_9.json
Partie 10 sauvegardée sous dossier_decoupes/partie_10.json
Partie 11 sauvegardée sous dossier_decoupes/partie_11.json
Partie 12 sauvegardée sous dossier_decoupes/partie_12.json
Partie 13 sauvegardée sous dossier_decoupes/partie_13.json
Partie 14 sauvegardée sous dossier_decoupes/partie_14.json
Partie 15 sauvegardée sous dossier_decoupes/partie_15.json
Partie 16 sauvegardée sous dossier_decoupes/partie_16.json
Partie 17 sauvegardée sous dossier_decoupes/partie_17.json
Partie 18 sauve

In [ ]:
def corriger_format_json(nom_fichier_source, fichier_sortie):
    # Lire les lignes du fichier source
    with open(nom_fichier_source, 'r', encoding='utf-8') as fichier_source:
        lignes = fichier_source.readlines()

    # Ajouter une virgule à chaque ligne sauf la dernière
    lignes = [ligne.strip() + ',' if i < len(lignes) - 1 else ligne.strip()
              for i, ligne in enumerate(lignes)]

    # Encapsuler les lignes dans un tableau JSON
    contenu = '[\n' + '\n'.join(lignes) + '\n]'

    # Écrire le contenu corrigé dans le fichier de sortie
    with open(fichier_sortie, 'w', encoding='utf-8') as fichier_sortie:
        fichier_sortie.write(contenu)

    print(f"Le fichier {nom_fichier_source} a été corrigé et sauvegardé sous {fichier_sortie}")

corriger_format_json("train.json", "train_corrige.json")


Le fichier train.json a été corrigé et sauvegardé sous <_io.TextIOWrapper name='train_corrige.json' mode='w' encoding='utf-8'>


In [ ]:
# Étape 1 : Corriger le fichier JSON
corriger_format_json("partie1.json", "partie1_corrige.json")

Le fichier partie1.json a été corrigé et sauvegardé sous <_io.TextIOWrapper name='partie1_corrige.json' mode='w' encoding='utf-8'>


In [ ]:
import json
import re

def corriger_json(fichier_source, fichier_sortie):
    """
    Corrige les séquences d'échappement invalides dans un fichier JSON.
    :param fichier_source: Chemin du fichier JSON source.
    :param fichier_sortie: Chemin du fichier JSON corrigé.
    """
    with open(fichier_source, 'r', encoding='utf-8') as fichier:
        contenu = fichier.read()

    # Corriger les séquences d'échappement invalides
    contenu_corrige = re.sub(r'\b', '', contenu)  # Retirer les '\b'
    contenu_corrige = contenu_corrige.replace('\\/', '/')  # Corriger les '\/'

    # Vérifier si le contenu corrigé est un JSON valide
    try:
        json.loads(f"[{contenu_corrige.strip()}]")
    except json.JSONDecodeError as e:
        print(f"Erreur de validation JSON : {e}")
        return

    # Sauvegarder le JSON corrigé
    with open(fichier_sortie, 'w', encoding='utf-8') as fichier:
        fichier.write(contenu_corrige)

    print(f"Fichier JSON corrigé sauvegardé sous : {fichier_sortie}")

# Exemple d'utilisation
fichier_source = "fichier_source.json"  # Chemin du fichier JSON source
fichier_sortie = "fichier_corrige.json"  # Chemin du fichier corrigé

corriger_json(fichier_source, fichier_sortie)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile

def corriger_format_json_dossier(dossier_source, dossier_sortie):
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(dossier_sortie):
        os.makedirs(dossier_sortie)
        # print("bonjour")

    # Parcourir tous les fichiers dans le dossier source
    for nom_fichier in os.listdir(dossier_source):
        chemin_fichier_source = os.path.join(dossier_source, nom_fichier)
        # print("bonjour")
        # Vérifier si c'est un fichier JSON
        if os.path.isfile(chemin_fichier_source) and nom_fichier.endswith('.json'):
            chemin_fichier_sortie = os.path.join(dossier_sortie, f"corrige_{nom_fichier}")
            # print("bonjour")
            # Lire les lignes du fichier source
            with open(chemin_fichier_source, 'r', encoding='utf-8') as fichier_source:
                lignes = fichier_source.readlines()

            # Ajouter une virgule à chaque ligne sauf la dernière
            lignes = [ligne.strip() + ',' if i < len(lignes) - 1 else ligne.strip()
                      for i, ligne in enumerate(lignes)]

            # Encapsuler les lignes dans un tableau JSON
            contenu = '[\n' + '\n'.join(lignes) + '\n]'

            # Écrire le contenu corrigé dans le fichier de sortie
            with open(chemin_fichier_sortie, 'w', encoding='utf-8') as fichier_sortie:
                fichier_sortie.write(contenu)

            print(f"Le fichier {nom_fichier} a été corrigé et sauvegardé sous {chemin_fichier_sortie}")

# Exemple d'utilisation
corriger_format_json_dossier("/content/dossier_sortie_1", "Data_hajar_corrected")


Le fichier dev.json a été corrigé et sauvegardé sous Data_hajar_corrected/corrige_dev.json


In [ ]:
import shutil

def zipper_dossier(dossier_source, fichier_zip):
    """
    Compresse un dossier en un fichier ZIP.

    :param dossier_source: Chemin du dossier à compresser.
    :param fichier_zip: Nom du fichier ZIP de sortie (sans extension).
    """
    # Créer un fichier zip à partir du dossier
    shutil.make_archive(fichier_zip, 'zip', dossier_source)
    print(f"Dossier {dossier_source} compressé en {fichier_zip}.zip")

# Exemple d'utilisation
dossier_source = "/content/new"  # Remplacez par le chemin de votre dossier
fichier_zip = "/content/none_done_by_matecat"    # Remplacez par le chemin souhaité pour le fichier ZIP

zipper_dossier(dossier_source, fichier_zip)

# Télécharger le fichier ZIP
from google.colab import files
files.download(f"{fichier_zip}.zip")


Dossier /content/new compressé en /content/none_done_by_matecat.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#reformat dev and test

In [ ]:
import json

def reformat_data_from_file(input_file, output_file):
    """
    Lit les données d'un fichier JSON, les reformate, et les sauvegarde au format JSONL.

    :param input_file: Chemin du fichier JSON contenant les données d'entrée.
    :param output_file: Chemin du fichier JSONL où les données reformattées seront sauvegardées.
    """
    formatted_data = []

    with open(input_file, "r", encoding="utf-8") as f:
        # Lire le fichier ligne par ligne
        for line in f:
            try:
                # Essayer de charger la ligne comme un objet JSON
                item = json.loads(line)

                # Reformater l'élément des données
                exp_content = item.get("exp")
                formatted_data.append({
                    "messages": [
                        {
                            "role": "user",
                            "content": (
                                f"Question : {item['question']}\n\n"
                                f"Options :\n"
                                f"a) {item.get('opa')}\n"
                                f"b) {item.get('opb')}\n"
                                f"c) {item.get('opc')}\n"
                                f"d) {item.get('opd')}\n\n"
                                f"Sujet : {item.get('subject_name')}\n"
                                f"topic_name : {item.get('topic_name')}\n"
                                f"id : {item.get('id')}\n"
                                f"choice_type : {item.get('choice_type')}"
                            )
                        }
                    ]
                })

            except json.JSONDecodeError as e:
                # Gérer les erreurs de décodage JSON, par exemple, ignorer la ligne ou afficher un message d'erreur
                print(f"Erreur de décodage JSON sur la ligne: {line.strip()} - Erreur: {e}")

    # Sauvegarder les données reformattées au format JSONL
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in formatted_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Les données formatées ont été sauvegardées dans '{output_file}'.")


# Exemple d'utilisation
input_file = "/content/dev.json"  # Chemin vers le fichier JSON d'entrée
output_file = "dev.jsonl"  # Chemin vers le fichier JSONL de sortie
reformat_data_from_file(input_file, output_file)

#reformat train

In [ ]:
import json

def reformat_data_from_file(input_file, output_file):
    """
    Lit les données d'un fichier JSON, les reformate, et les sauvegarde au format JSONL.

    :param input_file: Chemin du fichier JSON contenant les données d'entrée.
    :param output_file: Chemin du fichier JSONL où les données reformattées seront sauvegardées.
    """
    # Lire le fichier JSON
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    formatted_data = []

    for item in data:
        # Reformater chaque élément des données
        # exp_content = item.get("exp")
        formatted_data.append({
            "messages": [
                {
                    "role": "user",
                    "content": item.get('question')#(
                        # f"Question : {item.get('question')}\n\n"
                        # f"Options :\n"
                        # f"opa : {item.get('opa')}\n"
                        # f"opb : {item.get('opb')}\n"
                        # f"opc : {item.get('opc')}\n"
                        # f"opd : {item.get('opd')}\n\n"

                        # f"Thématique :\n"
                        # f"Sujet : {item.get('nom_du_sujet')}\n"
                        # f"nom_du_thème : {item.get('nom_du_thème')}\n"
                        # f"id : {item.get('id')}\n"
                        # f"choix : {item.get('type_de_choix')}"
                    # )
                },
                {
                    "role": "assistant",
                    "content": item.get("exp")
                }
            ]
        })

    # Sauvegarder les données reformattées au format JSON
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in formatted_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"Les données formatées ont été sauvegardées dans '{output_file}'.")

# Exemple d'utilisation
input_file = "/content/train.json"  # Chemin vers le fichier JSON d'entrée
output_file = "reformated_train.json"  # Chemin vers le fichier JSONL de sortie
reformat_data_from_file(input_file, output_file)


Les données formatées ont été sauvegardées dans 'reformated_train.json'.


#combine json

In [ ]:
import os
import json

def combine_json_files(input_folder, output_file):
    """
    Combine tous les fichiers JSON d'un dossier en un seul fichier JSON.

    :param input_folder: Chemin du dossier contenant les fichiers JSON.
    :param output_file: Chemin du fichier JSON combiné.
    """
    combined_data = []

    # Parcourir tous les fichiers du dossier
    for filename in os.listdir(input_folder):
        if filename.endswith(".json"):
            file_path = os.path.join(input_folder, filename)

            # Lire et charger chaque fichier JSON
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                    # Ajouter les données au tableau combiné
                    if isinstance(data, list):
                        combined_data.extend(data)
                    elif isinstance(data, dict):
                        combined_data.append(data)
                    else:
                        print(f"Format inconnu dans le fichier : {filename}")

            except json.JSONDecodeError as e:
                print(f"Erreur de décodage JSON dans le fichier {filename}: {e}")

    # Écrire les données combinées dans le fichier de sortie
    with open(output_file, "w", encoding="UTF-8") as f:
        json.dump(combined_data, f, ensure_ascii=False, indent=4)

    print(f"Les fichiers JSON ont été combinés dans '{output_file}'.")

# Exemple d'utilisation
input_folder = "/content/drive/MyDrive/translated_final"  # Remplacez par le chemin de votre dossier JSON
output_file = "/content/drive/MyDrive/train_js.json"    # Nom du fichier de sortie
combine_json_files(input_folder, output_file)


Les fichiers JSON ont été combinés dans '/content/drive/MyDrive/train_js.json'.


#reformat code de base

In [ ]:
import json

# Exemple de dataset brut
data = [
    {
        "question": "L'obstruction uréthrique chronique due à une hyperplasie prismatique bénigne peut entraîner le changement suivant du parenchyme rein",
        "exp": "L'obstruction uréthrique chronique à cause des calculi urinaires, de la hyperophy, des tumeurs, de la grossesse normale, des tumeurs, de la prolifération utérine ou des troubles fonctionnels cause une hydronéphrose qui, par définition, est utilisée pour décrire la dilatation du bassin rénal et du calcul associé atrophie progressive du rein en raison de l'obstruction de l'écoulement de l'urine Voir Robbins 7yh/9,1012,9/f.",
        "opa": "Hyperplasie",
        "opb": "Hyperophyse",
        "opc": "Atrophie",
        "opd": "Dyplasie",
        "subject_name": "Anatomie",
        "topic_name": "Traitement urinaire",
        "id": "e9ad821a-c438-4965-9f77-760819dfa155",
        "choice_type": "et"
    },
    {
        "question": "Quelle vitamine est fournie à partir d'une seule source animale :",
        "exp": "Ans. c) Vitamine B12 Ref: Harrison's 19th ed. P 640* La vitamine B12 (Cobalamin) est synthétisée uniquement par des micro-organismes*. Chez les humains, la seule source d'origine animale est la nourriture d'origine animale, p. ex. la viande, le poisson et les produits laitiers*. L'origine non animale ne contient pas de vitamine B12 .* Les exigences quotidiennes de vitamine Bp sont d'environ 1 à 3 pg. Les magasins de corps sont de l'ordre de 2-3 mg, suffisants pour 3 à 4 ans si les fournitures sont complètement coupées.",
        "opa": "Vitamine C",
        "opb": "Vitamine B7",
        "opc": "Vitamine B12",
        "opd": "Vitamine D",
        "subject_name": "Biochimie",
        "topic_name": "Vitamines et minéraux",
        "id": "e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3",
        "choice_type": "et"
    }
]

# Fonction pour reformater les données
def reformat_data(data):
    formatted_data = []

    for item in data:
        # Contexte system (peut être omis si non nécessaire)
        formatted_data.append({
            "messages": [
                {
                    "role": "user",
                    "content": (
                        f"Explication : {item['question']}\n\n"
                        f"Options :\n"
                        f"a) {item['opa']}\n"
                        f"b) {item['opb']}\n"
                        f"c) {item['opc']}\n"
                        f"d) {item['opd']}\n"
                        f"a) {item['subject_name']}\n"
                        f"b) {item['topic_name']}\n"
                        f"c) {item['id']}\n"
                        f"d) {item['choice_type']}"
                    )
                },
                {
                    "role": "assistant",
                    "content": item['exp'],
                }
            ]
        })

    return formatted_data

# Reformater les données
formatted_data = reformat_data(data)

In [ ]:
import pandas as pd

data = pd.DataFrame(data)

In [ ]:
data

,question,exp,opa,opb,opc,opd,subject_name,topic_name,id,choice_type
0,L'obstruction uréthrique chronique due à une h...,L'obstruction uréthrique chronique à cause des...,Hyperplasie,Hyperophyse,Atrophie,Dyplasie,Anatomie,Traitement urinaire,e9ad821a-c438-4965-9f77-760819dfa155,et
1,Quelle vitamine est fournie à partir d'une seu...,Ans. c) Vitamine B12 Ref: Harrison's 19th ed. ...,Vitamine C,Vitamine B7,Vitamine B12,Vitamine D,Biochimie,Vitamines et minéraux,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,et


In [ ]:
def clean_newlines(text):
    return re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

french_wikipedia_prompt = """
titre : {}
question : {}
explication : {}
opa : {}
opb : {}
opc : {}
opd : {}
subject_name : {}
topic_name : {}
id : {}
choice_type : {}
text : {}
"""

EOS_TOKEN = tokenizer.eos_token  # Doit ajouter l'EOS_TOKEN

def french_formatting_prompts_func(examples):
    # Récupérer les informations pertinentes
    questions = examples.get("question", [])
    exps = examples["exp"]
    opas = examples["opa"]
    opbs = examples["opb"]
    opc = examples["opc"]
    opds = examples["opd"]
    subject_names = examples["subject_name"]
    topic_names = examples["topic_name"]
    ids = examples["id"]
    choice_types = examples["choice_type"]

    outputs = []

    for question, exp, opa, opb, opc, opd, subject_name, topic_name, id, choice_type in zip(
        questions, exps, opas, opbs, opc, opds, subject_names, topic_names, ids, choice_types
    ):
        # Construire le texte formaté pour chaque exemple
        formatted_text = french_wikipedia_prompt.format(
            question,
            question,  # ici, on réutilise `question` pour le titre
            clean_newlines(exp),  # Nettoyer les sauts de ligne dans l'explication
            opa,
            opb,
            opc,
            opd,
            subject_name,
            topic_name,
            id,
            choice_type,
            exp  # Le texte ici est basé sur "exp"
        ) + EOS_TOKEN  # Ajouter l'EOS_TOKEN pour la génération de texte

        outputs.append(formatted_text)

    return {"text": outputs}


In [ ]:
formatted_data

[{'messages': [{'role': 'user',
    'content': "Explication : L'obstruction uréthrique chronique due à une hyperplasie prismatique bénigne peut entraîner le changement suivant du parenchyme rein\n\nOptions :\na) Hyperplasie\nb) Hyperophyse\nc) Atrophie\nd) Dyplasie\na) Anatomie\nb) Traitement urinaire\nc) e9ad821a-c438-4965-9f77-760819dfa155\nd) et"},
   {'role': 'assistant',
    'content': "L'obstruction uréthrique chronique à cause des calculi urinaires, de la hyperophy, des tumeurs, de la grossesse normale, des tumeurs, de la prolifération utérine ou des troubles fonctionnels cause une hydronéphrose qui, par définition, est utilisée pour décrire la dilatation du bassin rénal et du calcul associé atrophie progressive du rein en raison de l'obstruction de l'écoulement de l'urine Voir Robbins 7yh/9,1012,9/f."}]},
 {'messages': [{'role': 'user',
    'content': "Explication : Quelle vitamine est fournie à partir d'une seule source animale :\n\nOptions :\na) Vitamine C\nb) Vitamine B7\nc)

In [ ]:
import pandas as pd
import json

# Charger le fichier JSON
with open("reformated_train.json", "r", encoding="utf-8") as f:
    all_data = json.load(f)

# Accéder à la clé "messages" et transformer en DataFrame
data = pd.json_normalize(data["messages"])
df = pd.concat([pd.DataFrame(message) for message in data["messages"]], ignore_index=True)


In [ ]:
import pandas as pd
import json

# Charger le fichier JSONL ligne par ligne
all_data = []
with open("reformated_train.json", "r", encoding="utf-8") as f:
    for line in f:
        try:
            all_data.append(json.loads(line))  # Charger chaque ligne comme un objet JSON
        except json.JSONDecodeError as e:
            print(f"Erreur lors du chargement de la ligne : {line.strip()} - Erreur : {e}")

df = pd.DataFrame(all_data)


In [ ]:
df = pd.DataFrame(all_data)

In [ ]:
df['messages'][1]

[{'role': 'user',
  'content': 'Question : La teneur en sodium de ReSoMal est\n\nOptions :\na) 90 mmol/lit\nb) 60 mmol/lit\nc) 45 mmol/lit\nd) 30 mmol/lit\n\nThématique :\nSujet : Pédiatrie\nnom_du_thème : Fluides et électrolytes\nid : 27aa38b1-1332-416a-9b83-27d6b78c6183\nchoix : et'},
 {'role': 'assistant',
  'content': 'Les poissons sont naturellement faibles en sodium et même les espèces ayant les niveaux de sodium les plus élevés contiennent moins de 100 milligrammes par 3oz de poion cuit. La plupart des mollusques ont généralement plus de sodium, allant de 100 à 500 milligrammes par 3oz de portions cuites. GHAI Essential Pediatrics, 8e édition'}]

In [ ]:
import pandas as pd
df = pd.DataFrame(formatted_data)
df

,messages
0,"[{'role': 'user', 'content': 'Explication : L'..."
1,"[{'role': 'user', 'content': 'Explication : Qu..."


#Translation

In [ ]:
!git clone https://huggingface.co/SLPG/Biomedical_MT_EN-FR


Cloning into 'Biomedical_MT_EN-FR'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 7), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (19/19), 246.18 KiB | 2.24 MiB/s, done.


In [ ]:
!apt-get update && apt-get install -y build-essential libffi-dev libssl-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,226 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,196 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,448 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 htt

In [ ]:
!pip install --upgrade setuptools wheel cython

In [ ]:
!pip install pip==23.2.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 34.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for hydra-core<1.1,>=1.0.7 from https://files.pythonhosted.org/packages/5f/2a/9c698daa12ed6e09e7629e6908528f043fa9de8a441c56cc13608d765fb2/hydra_core-1.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu>=1.4.12 from https://files.pythonhosted.org/packages/15/d8/e51d35bc863caa19ddeae48dfb890581a19326973ad1c9fa5dcfc63310f7/sacrebleu-2.4.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.8 MB/s eta 0:00:00
  Obtaining dependency infor

In [ ]:
from fairseq.models.transformer import TransformerModel


In [ ]:
!wget https://huggingface.co/SLPG/Biomedical_MT_EN-FR/resolve/main/pytorch_model.bin -O /content/Biomedical_MT_EN-FR/model.pt

In [ ]:
!pip install sentencepiece

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 42.4 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline



device = 0 if torch.cuda.is_available() else -1
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-fr",device=device, batch_size = 16)
# translator = pipeline(task="translation_es_to_fr", model="t5-base", device=device)
# translator = pipeline(task="translation_en_to_fr", model="t5-small", device=device)

translator("¡Me encanta Hajar!")[0]["translation_text"]

Device set to use cuda:0


"J'adore Hajar !"

In [ ]:
translator("""ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
Para ello ponemos a tu disposición:
Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
Programa La Salud Inteligente de Adeslas
Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
El 35% de la población en España respira aire contaminado
Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
Identifícate en la cabecera de arriba con tu usuario y contraseña.
Si no dispones de ellos, date de alta aquí
Fumar afecta también a la piel
Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
""")

[{'translation_text': "ATTENTION: CE PAGE EST DÉPROTÉ OU EN MODE ÉDITATION Adeslas vous aide à préserver et à améliorer votre santé tout au long de la vie. Pour ce faire, nous mettons à votre disposition: Informations et outils pour la prévention des risques et des maladies, ainsi que des plans de soins pour récupérer le bien-être en cas de maladie chronique ou d'étape spéciale de la vie. Programme La santé intelligente d'Adeslas Dirigé aux personnes souffrant de diabète, de maladie cardiovasculaire (infarctus ou angine de poitrine), ayant des antécédents ou des facteurs de risque (tabac, glucose ou cholestérol élevé, hypertension artérielle ou surpoids). 35% de la population en Espagne respire de l'air pollué Selon l'Institut national de statistique (INE) et le Ministère de l'environnement, 3% de la mortalité annuelle en Espagne est due à la pollution atmosphérique. Identifiez-vous dans la tête d'hôtel avec votre utilisateur et votre mot de passe. Si vous n'en avez pas, datez ici Fume

In [ ]:
!pip install transformers

In [ ]:
import os
import json
from transformers import pipeline
import torch

# Charger le modèle Helsinki avec GPU si disponible
device = 0 if torch.cuda.is_available() else -1
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr", device=device)

def split_text(text, max_length=512):
    """Divise un texte en morceaux pour respecter la limite de longueur du modèle."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # Inclure l'espace
        if current_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def translate_text(text):
    """Traduit un texte tout en gérant les longues chaînes."""
    chunks = split_text(text, max_length=512)
    translated_chunks = [translator(chunk)[0]["translation_text"] for chunk in chunks]
    return " ".join(translated_chunks)

def translate_file(input_file, output_file):
    """Traduit un fichier JSON ligne par ligne."""
    with open(input_file, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    # Assurez-vous que les données sont une liste de dictionnaires
    if not isinstance(data, list):
        raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

    translated_data = []
    for item in data:
        translated_item = {}
        for key, value in item.items():
            if isinstance(value, str):  # Traduire uniquement les chaînes de caractères
                translated_value = translate_text(value)
                translated_item[key] = translated_value
            else:
                translated_item[key] = value  # Conserver les autres types de données
        translated_data.append(translated_item)

    # Écrire les données traduites dans le fichier de sortie
    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(translated_data, outfile, ensure_ascii=False, indent=4)

def translate_directory(input_dir, output_dir):
    """Traduit tous les fichiers JSON d'un répertoire."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".json"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, filename)
            print(f"Traduction du fichier : {input_file}")
            translate_file(input_file, output_file)

# Spécifiez les chemins des répertoires d'entrée et de sortie
input_directory = "/content/drive/MyDrive/New folder"
output_directory = "/content/new"

# Traduire tous les fichiers du répertoire
translate_directory(input_directory, output_directory)


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# import os
# import json
# from datasets import Dataset
# from transformers import pipeline

# def split_text(text, max_length=267):
#     """Divise un texte en morceaux pour respecter la limite de longueur."""
#     words = text.split()
#     chunks = []
#     current_chunk = []
#     current_length = 0

#     for word in words:
#         current_length += len(word) + 1  # Inclure l'espace
#         if current_length > max_length:
#             chunks.append(" ".join(current_chunk))
#             current_chunk = [word]
#             current_length = len(word) + 1
#         else:
#             current_chunk.append(word)

#     if current_chunk:
#         chunks.append(" ".join(current_chunk))
#     return chunks

# def map_text_with_indices(data, max_length):
#     """Mappe le texte en morceaux avec des indices uniques."""
#     mapped_data = []
#     for idx, item in enumerate(data):
#         for key, value in item.items():
#             if isinstance(value, str):
#                 chunks = split_text(value, max_length=max_length)
#                 for chunk_idx, chunk in enumerate(chunks):
#                     mapped_data.append({"index": f"{idx}-{key}-{chunk_idx}", "text": chunk})
#     return mapped_data

# def recombine_chunks(data, translated_chunks):
#     """Recombine les morceaux traduits dans leur format d'origine."""
#     index_map = {}
#     for chunk in translated_chunks:
#         idx, text = chunk["index"], chunk["text"]
#         main_idx, key, _ = idx.split("-")
#         main_idx = int(main_idx)

#         if main_idx not in index_map:
#             index_map[main_idx] = {}

#         if key not in index_map[main_idx]:
#             index_map[main_idx][key] = []

#         index_map[main_idx][key].append(text)

#     combined_data = []
#     for idx, items in index_map.items():
#         reconstructed_item = {}
#         for key, texts in items.items():
#             reconstructed_item[key] = " ".join(texts)
#         combined_data.append(reconstructed_item)

#     return combined_data

# def translate_with_pipeline(data, batch_size=16):
#     """Traduit les textes par lots avec un pipeline."""
#     translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

#     dataset = Dataset.from_list(data)
#     translated_chunks = dataset.map(
#         lambda examples: {"text": translator(examples["text"], max_length=267)[0]["translation_text"]},
#         batched=True,
#         batch_size=batch_size
#     )
#     return translated_chunks

# def translate_file(input_file, output_file, max_length=267, batch_size=16):
#     """Traduit un fichier JSON avec gestion des gros contenus."""
#     with open(input_file, "r", encoding="utf-8") as infile:
#         data = json.load(infile)

#     if not isinstance(data, list):
#         raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

#     mapped_data = map_text_with_indices(data, max_length=max_length)
#     translated_chunks = translate_with_pipeline(mapped_data, batch_size=batch_size)
#     recombined_data = recombine_chunks(data, translated_chunks)

#     with open(output_file, "w", encoding="utf-8") as outfile:
#         json.dump(recombined_data, outfile, ensure_ascii=False, indent=4)

# def translate_directory(input_dir, output_dir, max_length=267, batch_size=16):
#     """Traduit tous les fichiers JSON dans un répertoire."""
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for filename in os.listdir(input_dir):
#         if filename.endswith(".json"):
#             input_file = os.path.join(input_dir, filename)
#             output_file = os.path.join(output_dir, filename)
#             print(f"Traduction du fichier : {input_file}")
#             translate_file(input_file, output_file, max_length=max_length, batch_size=batch_size)

# # Spécifiez les chemins des répertoires d'entrée et de sortie
# input_directory = "/content/drive/MyDrive/New folder"
# output_directory = "/content/new"

# # Traduire tous les fichiers du répertoire
# translate_directory(input_directory, output_directory)


In [ ]:
data = map_text_with_indices('/content/corrige_partie_1.json', 250)

In [ ]:
# If you just want the content as a single string:
with open('/content/drive/MyDrive/CoWeSe.txt', 'r') as file:
    data = file.read()

In [ ]:
import os
import json
from datasets import Dataset
from transformers import pipeline

def split_text(text, max_length=267):
    """Divise un texte en morceaux pour respecter la limite de longueur."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # Inclure l'espace
        if current_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def map_text_with_indices(file_path, max_length):
    """Mappe le texte en morceaux avec des indices uniques."""
    with open(file_path, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    if not isinstance(data, list):
        raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

    mapped_data = []
    for idx, item in enumerate(data):
        for key, value in item.items():
            if isinstance(value, str):
                chunks = split_text(value, max_length=max_length)
                for chunk_idx, chunk in enumerate(chunks):
                    mapped_data.append({"index": f"{idx}-{key}-{chunk_idx}", "text": chunk})
    return mapped_data

# Exemple d'utilisation
# data = map_text_with_indices('/content/corrige_partie_1.json', 250)

In [ ]:
data

[{'index': '0-question-0',
  'text': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma'},
 {'index': '0-exp-0',
  'text': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and'},
 {'index': '0-exp-1',
  'text': 'calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950'},
 {'index': '0-opa-0', 'text': 'Hyperplasia'},
 {'index': '0-opb-0', 'text': 'Hyperophy'},
 {'index': '0-opc-0', 'text': 'Atrophy'},
 {'index': '0-opd-0', 'text': 'Dyplasia'},
 {'index': '0-subject_name-0', 'text': 'Anatomy'},
 {'index': '0-topic_name-0', 'text': 'Urinary tract'},
 {'index': '0-id-0', 'text': 'e9ad821a-c438-4965-9f77-760819dfa155'},
 {'index': '0-choice_type-0', 'text

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.3 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline(task="translation_en_to_fr", model="t5-small", batch_size=batch_size, device=device)

    dataset = Dataset.from_list(data)

    def translate_batch(batch):
        # translations = translator(batch["text"], max_length=267)
        translations = translator(KeyDataset(dataset, "text"), batch_size, truncation="only_first")

        return {"text": [translation["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks

# Exemple d'utilisation
# data = [{"text": "This is a test sentence."}, {"text": "Another example."}]
# translated_chunks = translate_with_pipeline(data, batch_size=16)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
def recombine_chunks(data, translated_chunks):
    """Recombine les morceaux traduits dans leur format d'origine."""
    index_map = {}
    for chunk in translated_chunks:
        idx, text = chunk["index"], chunk["text"]
        main_idx, key, _ = idx.split("-")
        main_idx = int(main_idx)

        if main_idx not in index_map:
            index_map[main_idx] = {}

        if key not in index_map[main_idx]:
            index_map[main_idx][key] = []

        index_map[main_idx][key].append(text)

    combined_data = []
    for idx, items in index_map.items():
        reconstructed_item = {}
        for key, texts in items.items():
            reconstructed_item[key] = " ".join(texts)
        combined_data.append(reconstructed_item)

    return combined_data

In [ ]:
translated_data = recombine_chunks(data, translated_chunks)

In [ ]:
translated_data

[{'question': "L'obstruction uréthrique chronique due à une hyperplasie prismatique bénigne peut entraîner le changement suivant du parenchyme rein",
  'exp': "L'obstruction uréthrique chronique en raison de calculs urinaires, de l'hypotrophie prostatique, des tumeurs, de la grossesse normale, des tumeurs, de la prolifération utérine ou des troubles fonctionnels cause une hydronéphrose qui, par définition, est utilisée pour décrire la dilatation du bassin rénal et calculus associé à l'atrophie progressive du rein en raison de l'obstruction de l'écoulement de l'urine Refer Robbins 7yh/9,1012,9/f. P950",
  'opa': 'Hyperplasie',
  'opb': 'Hyperophyse',
  'opc': 'Atrophie',
  'opd': 'Dyplasie',
  'subject_name': 'Anatomie',
  'topic_name': 'Traitement urinaire',
  'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
  'choice_type': 'et'},
 {'question': "Quelle vitamine est fournie à partir d'une seule source animale :",
  'exp': "Ans. c) Vitamine B12 Ref: Harrison's 19th ed. P 640* La vitamine B

In [ ]:
def translate_file(input_file, output_file, max_length=267, batch_size=32):
    """Traduit un fichier JSON avec gestion des gros contenus."""
    with open(input_file, "r", encoding="utf-8") as infile:
        data = json.load(infile)

    if not isinstance(data, list):
        raise ValueError("Le fichier JSON doit contenir une liste d'objets.")

    mapped_data = map_text_with_indices(input_file, max_length=max_length)
    translated_chunks = translate_with_pipeline(mapped_data, batch_size=batch_size)
    recombined_data = recombine_chunks(data, translated_chunks)

    with open(output_file, "w", encoding="utf-8") as outfile:
        json.dump(recombined_data, outfile, ensure_ascii=False, indent=4)

def translate_directory(input_dir, output_dir, max_length=267, batch_size=32):
    """Traduit tous les fichiers JSON dans un répertoire."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".json"):
            input_file = os.path.join(input_dir, filename)
            output_file = os.path.join(output_dir, filename)
            print(f"Traduction du fichier : {input_file}")
            translate_file(input_file, output_file, max_length=max_length, batch_size=batch_size)

# Spécifiez les chemins des répertoires d'entrée et de sortie
input_directory = "/content/drive/MyDrive/New folder"
output_directory = "/content/new"

# Traduire tous les fichiers du répertoire
translate_directory(input_directory, output_directory)

#Pyspark

In [ ]:
!pip install pyspark

In [ ]:
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType # Import the necessary classes


In [ ]:
# Créer une session Spark
spark = SparkSession.builder.appName("Cas d'usage DVF").config("spark.master", "local[4]").getOrCreate()

In [ ]:
# spark is from the previous example
sc = spark.sparkContext

# A text dataset is pointed to by path.
# The path can be either a single text file or a directory of text files
path = "/content/drive/MyDrive/CoWeSe.txt"

df1 = spark.read.text(path)
df1 = df1.toDF("texte")
# df1.show()
# # +-----------+
# # |      value|
# # +-----------+
# # |Michael, 29|
# # |   Andy, 30|
# # | Justin, 19|
# # +-----------+

# # You can use 'lineSep' option to define the line separator.
# # The line separator handles all `\r`, `\r\n` and `\n` by default.
# df2 = spark.read.text(path, lineSep=",")
# df2.show()
# # +-----------+
# # |      value|
# # +-----------+
# # |    Michael|
# # |   29\nAndy|
# # | 30\nJustin|
# # |       19\n|
# # +-----------+

# # You can also use 'wholetext' option to read each input file as a single row.
# df3 = spark.read.text(path, wholetext=True)
# df3.show()
# # +--------------------+
# # |               value|
# # +--------------------+
# # |Michael, 29\nAndy...|
# # +--------------------+

# # "output" is a folder which contains multiple text files and a _SUCCESS file.
# df1.write.csv("output")

# # You can specify the compression format using the 'compression' option.
# df1.write.text("output_compressed", compression="gzip")

In [ ]:
df1.show()

+--------------------+
|               texte|
+--------------------+
|ATENCIÓN: ESTA PÁ...|
|Adeslas te ayuda ...|
|Para ello ponemos...|
|Información y her...|
|Programa La Salud...|
|Dirigido a person...|
|El 35% de la pobl...|
|Según el Institut...|
|Identifícate en l...|
|Si no dispones de...|
|Fumar afecta tamb...|
|Una razón más par...|
|                    |
|ATENCIÓN: ESTA PÁ...|
|El Portal de Plan...|
|Si detecta dificu...|
|Si detecta dificu...|
|En caso contrario...|
|Si detecta dificu...|
|                    |
+--------------------+
only showing top 20 rows



In [ ]:
# 2. Calculer le nombre total de lignes
total_rows = df1.count()
print(f"Nombre total de lignes : {total_rows}")

# 3. Déterminer la taille approximative de chaque partition
partition_size = total_rows // 100
print(f"Taille approximative par partition : {partition_size}")

Nombre total de lignes : 34347553
Taille approximative par partition : 343475


In [ ]:
# 4. Repartitionner les données en 100 partitions
df_partitioned = df1.repartition(100)

# 5. Sauvegarder les partitions dans un dossier
output_path = "/content/drive/MyDrive/CoWeSe_partitions_2"
# Désactiver la génération des fichiers .crc
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

# Sauvegarder les partitions
# df_partitioned.write.text(output_path)

df_partitioned.write.text(output_path)

print(f"Les données ont été sauvegardées dans : {output_path}")

Les données ont été sauvegardées dans : /content/drive/MyDrive/CoWeSe_partitions_2


In [ ]:
# !pip install shutil
import shutil
from google.colab import files

def zipper_dossier(dossier_source, fichier_zip):
    """
    Compresses a folder into a ZIP file.

    :param dossier_source: Path of the folder to compress.
    :param fichier_zip: Name of the output ZIP file (without extension).
    """
    # Create a zip file from the folder
    shutil.make_archive(fichier_zip, 'zip', dossier_source)
    print(f"Folder {dossier_source} compressed to {fichier_zip}.zip")


dossier_source = "/content/drive/MyDrive/CoWeSe_partitions"  # Replace with your folder path
fichier_zip = "CoWeSe_partitions"    # Name of the output zip file

zipper_dossier(dossier_source, fichier_zip)
files.download(f"{fichier_zip}.zip")

Folder /content/drive/MyDrive/CoWeSe_partitions compressed to CoWeSe_partitions.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os

def get_folder_size_in_gb(folder_path):
    total_size = 0
    # Parcourir tous les fichiers dans le dossier
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            # Ajouter la taille du fichier au total
            total_size += os.path.getsize(file_path)
    # Convertir en gigaoctets
    size_in_gb = total_size / (1024 ** 3)
    return size_in_gb

# Chemin du dossier
output_path = "/content/drive/MyDrive/CoWeSe_partitions"

# Calculer la taille
folder_size_gb = get_folder_size_in_gb(output_path)
print(f"La taille totale du dossier est : {folder_size_gb:.2f} GB")


La taille totale du dossier est : 4.45 GB


Lecture du fichier au format paquet

In [ ]:
import torch
# from datasets import Dataset
from transformers import pipeline
# from transformers.pipelines.pt_utils import KeyDataset

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1

# 1. Initialiser SparkSession
spark = SparkSession.builder \
    .appName("ParquetTranslation") \
    .getOrCreate()

# 2. Charger le fichier text
df_paquet = spark.read.text("/content/drive/MyDrive/CoWeSe_partitions")

# 3. Initialiser le modèle de traduction
translator = pipeline(task="translation_sp_to_fr", model="t5-small", batch_size=16, device=device)  # device=0 pour GPU, -1 pour CPU

# 4. Fonction de traduction
def translate_text(text):
    try:
        # Traduire le texte en utilisant le modèle
        result = translator(text, max_length=267)
        return result[0]['translation_text']
    except Exception as e:
        return str(e)

# 5. Enregistrer la fonction comme UDF (User Defined Function)
translate_udf = udf(translate_text, StringType())

# 6. Appliquer la traduction sur les données
df_translated = df_paquet.withColumn("translated_text", translate_udf(df_paquet["value"]))

# 7. Sauvegarder les résultats au format Parquet
df_translated.write.parquet("translated.parquet")

# 8. Afficher quelques résultats pour vérification
df_translated.show()


In [ ]:
df_paquet.show()

+--------------------+
|               value|
+--------------------+
|Indicadores estad...|
|Por su parte, Mer...|
|De modo que la hi...|
|Falta de adherenc...|
|Ramírez-Rodríguez...|
|Entre los determi...|
|algún tipo de pro...|
|La Ciudad de las ...|
|Adolescentes de b...|
|El perfeccionamie...|
|Antecedentes: Los...|
|Existen otras cau...|
|En cada sesión se...|
|A Alba Pérez Caña...|
|Significados para...|
|Montecino C. Las ...|
|En el recién naci...|
|más frecuentes, f...|
|Eritrosedimentaci...|
|Beldarraín Chaple...|
+--------------------+
only showing top 20 rows



In [ ]:
!pip install -qU langchain-text-splitters

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# This is a long document we can split up.
with open("/content/drive/MyDrive/CoWeSe_partitions/part-00000-2b585582-bcd5-4875-b49f-f8fd6f8f1e48-c000.txt") as f:
    texte = f.read()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=250,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
texts = text_splitter.create_documents([texte])
print(texts[0])
print(texts[1])

page_content='Por imagenología se encuentra un absceso gigante intraabdominal por lo que es llevada a cirugía laparoscópica con posterior drenaje exitoso de un absceso intraabdominal, con evolución satisfactoria.
El grado de lesión se evalúo con APACHE II y SOFA.'
page_content='Atención de enfermería en el trasplante de progenitores hematopoyéticos
Aunque para la Organización Mundial de la Salud (OMS), la morbimortalidad materna se ha convertido en un problema de primer orden y la prioridad es para la madre.'


In [ ]:
import torch
from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 15.6 MB/s eta 0:00:00


In [ ]:
# Diviser le texte en morceaux avec LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Charger le fichier texte

def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None
    # with open("/content/spark_tes.txt", "r", encoding="utf-8") as f:
    #     texte = f.read()

    if texte:
        # Diviser le texte en morceaux avec LangChain
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=400,          # Taille des morceaux
            chunk_overlap=0,        # Chevauchement entre morceaux
            length_function=len,
            is_separator_regex=False,
        )

        # Créer les documents
        documents = text_splitter.create_documents([texte])

    # Extraire les textes bruts des documents pour compatibilité avec Dataset
    data = [{"page_content": doc.page_content} for doc in documents]

    return data

# Traduire les morceaux avec le pipeline
import torch
from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset

def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-fr",device=device)
    # translator = pipeline(task="translation", model=model="Helsinki-NLP/opus-mt-es-fr", batch_size=batch_size, max_length=270, device=device)

    dataset = Dataset.from_list(data)  # Convertir la liste en Dataset Hugging Face

    def translate_batch(batch):

        translations = translator(KeyDataset(dataset, "page_content"), truncation="only_first")
        # print(translations)

        return {"page_content": [translation[0]["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks

# Exemple d'utilisation
file_path = "/content/spark_tes.txt"
data = load_data(file_path)
translated_chunks = translate_with_pipeline(data, batch_size=16)

# Afficher les résultats
print(translated_chunks)

Device set to use cpu


Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Dataset({
    features: ['page_content'],
    num_rows: 504
})


In [ ]:
device = 0 if torch.cuda.is_available() else -1
translator = pipeline(task="translation", model="Helsinki-NLP/opus-mt-es-fr", max_length=512, device=device)
translator(""" ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
              Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
              Para ello ponemos a tu disposición:
              Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
              Programa La Salud Inteligente de Adeslas
              Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
              El 35% de la población en España respira aire contaminado
              Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
              Identifícate en la cabecera de arriba con tu usuario y contraseña.
              Si no dispones de ellos, date de alta aquí
              Fumar afecta también a la piel
              Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
          """)

###Helsinki-NLP/opus-mt-es-fr

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/332M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/819k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

Device set to use cpu


[{'translation_text': "ATTENTION: CE PAGE EST DÉPROTÉ OU EN MODE ÉDITATION Adeslas vous aide à préserver et à améliorer votre santé tout au long de la vie. Pour ce faire, nous mettons à votre disposition: Informations et outils pour la prévention des risques et des maladies, ainsi que des plans de soins pour récupérer le bien-être en cas de maladie chronique ou d'étape spéciale de la vie. Programme La santé intelligente d'Adeslas Dirigé aux personnes souffrant de diabète, de maladie cardiovasculaire (infarctus ou angine de poitrine), ayant des antécédents ou des facteurs de risque (tabac, glucose ou cholestérol élevé, hypertension artérielle ou surpoids). 35% de la population en Espagne respire de l'air pollué Selon l'Institut national de statistique (INE) et le Ministère de l'environnement, 3% de la mortalité annuelle en Espagne est due à la pollution atmosphérique. Identifiez-vous dans la tête d'hôtel avec votre utilisateur et votre mot de passe. Si vous n'en avez pas, datez ici Fume

In [ ]:
from transformers import pipeline
import torch

# Set device
device = 0 if torch.cuda.is_available() else -1

# Load translation pipeline
translator = pipeline(
    task="translation",
    model="facebook/nllb-200-distilled-600M",
    max_length=512,
    device=device
)

# Define the text to be translated
text = """
ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
Para ello ponemos a tu disposición:
Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
Programa La Salud Inteligente de Adeslas
Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
El 35% de la población en España respira aire contaminado
Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
Identifícate en la cabecera de arriba con tu usuario y contraseña.
Si no dispones de ellos, date de alta aquí
Fumar afecta también a la piel
Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
"""

# Perform translation
translated_text = translator(text, src_lang="spa", tgt_lang="eng")

# Print translated text

translated_text[0]['translation_text']

Device set to use cpu


'eng: this page is protected or in a way of education. adhesives help you preserve and improve your health throughout life. for this we make available information and tools for the prevention of risks and diseases, as well as care plans to restore well-being in case of chronic illness or special stage of life. program of intelligent health adhesives aimed at people with diabetes, cardiovascular disease (heart attack or chest disease), with a history or risk factors (tobacco, glucose or cholesterol, hypertension or overweight). 35% of the population in Spain breathes contaminated air according to the National Institute of State (UN) and the Ministry of Environment, 3% of the annual user mortality in Spain should be high on the air pollution.'

In [ ]:
from transformers import pipeline
import torch

# Set device
device = 0 if torch.cuda.is_available() else -1

# Load translation pipeline
translator = pipeline(
    task="translation",
    model="facebook/nllb-200-3.3B",
    max_length=512,
    device=device
)

# Define the text to be translated
text = """
ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
Para ello ponemos a tu disposición:
Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
Programa La Salud Inteligente de Adeslas
Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
El 35% de la población en España respira aire contaminado
Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
Identifícate en la cabecera de arriba con tu usuario y contraseña.
Si no dispones de ellos, date de alta aquí
Fumar afecta también a la piel
Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
"""

# Perform translation (Spanish to French)
translated_text = translator(text, src_lang="spa", tgt_lang="eng")

# Print translated text
print(translated_text[0]['translation_text'])


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Device set to use cpu


engengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengengeng::::::::: AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT AT 

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."
article_es = """ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
                Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
                Para ello ponemos a tu disposición:
                Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
                Programa La Salud Inteligente de Adeslas
                Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
                El 35% de la población en España respira aire contaminado
                Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
                Identifícate en la cabecera de arriba con tu usuario y contraseña.
                Si no dispones de ellos, date de alta aquí
                Fumar afecta también a la piel
                Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
                """

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer.src_lang = "es_ES"
encoded_ar = tokenizer(article_es, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."


['In order to do this, we make available to you: Information and tools for the prevention of risks and diseases, as well as care plans to recover the well-being in the case of chronic diseases and special stages of life. The Intelligent Health of Yourself Program The Intelligent Health of Yourself Program Directed to people with diabetes, cardiovascular disease (infarction or angina), with a history of risk factors (smoking, glucose, high cholesterol, hypertension or overweight). 35% of the population in Spain breathes contaminated air According to the National Institute of Statistics (INE) and the Ministry of the Environment, 3% of the annual death rate in Spain is due to air pollution. Identify in the top header with your user and password. If you do not have them, date up here Smoking affects also the skin Smoking affects also the skin Another reason to stop smoking, the effects of']

In [ ]:
translated_chunks["page_content"]

["ATTENTION: CE PAGE EST DÉPROTÉ OU EN MODE EDITION Adeslas vous aide à préserver et à améliorer votre santé tout au long de votre vie. À cette fin, nous mettons à votre disposition des informations et des outils pour la prévention des risques et des maladies, ainsi que des plans de soins pour récupérer le bien-être en cas de maladie chronique ou d'étape spéciale de la vie.",
 "Il s'adresse aux personnes atteintes de diabète, de maladie cardiovasculaire (infarctus ou angine de poitrine), ayant des antécédents ou des facteurs de risque (tabac, glucose ou cholestérol élevé, hypertension artérielle ou surpoids). 35% de la population en Espagne respire de l'air contaminé.",
 "Selon l'Institut national de statistique (INE) et le Ministère de l'environnement, 3% de la mortalité annuelle en Espagne est due à la pollution atmosphérique. Identifiez-vous en haut avec votre utilisateur et votre mot de passe. Si vous n'en avez pas, la date de sortie ici Fumar affecte également la peau.",
 'Une rai

#Traduction de espagnol français d'un fichier

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import torch


# Charger le fichier texte
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None

    if not texte:
        print("Le fichier est vide.")
        return None

    # Diviser le texte en morceaux avec LangChain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Taille des morceaux
        chunk_overlap=0,  # Chevauchement entre morceaux
        length_function=len
    )

    # Créer les documents
    documents = text_splitter.create_documents([texte])

    # Extraire les textes bruts des documents pour compatibilité avec Dataset
    data = [{"page_content": doc.page_content} for doc in documents]

    return data


# Traduire les morceaux avec le pipeline
def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline("translation_es_to_fr", model="Helsinki-NLP/opus-mt-es-fr", device=device)

    dataset = Dataset.from_list(data)  # Convertir la liste en Dataset Hugging Face

    def translate_batch(batch):
        """Traduit un batch de textes."""
        translations = translator(batch["page_content"], truncation=True, max_length=512)
        return {"page_content": [translation["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks


# Exemple d'utilisation
file_path = "/content/spark_tes.txt"
data = load_data(file_path)

if data:
    translated_chunks = translate_with_pipeline(data, batch_size=16)
    print(translated_chunks)
else:
    print("Aucune donnée à traduire.")

Device set to use cpu


Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Dataset({
    features: ['page_content'],
    num_rows: 33
})


In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset
from transformers import pipeline
import torch


# Charger le fichier texte
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None

    if not texte:
        print("Le fichier est vide.")
        return None

    # Diviser le texte en morceaux avec LangChain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Taille des morceaux
        chunk_overlap=0,  # Chevauchement entre morceaux
        length_function=len
    )

    # Créer les documents
    documents = text_splitter.create_documents([texte])

    # Extraire les textes bruts des documents pour compatibilité avec Dataset
    data = [{"page_content": doc.page_content} for doc in documents]

    return data


# Traduire les morceaux avec le pipeline
def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-fr", device=device)

    dataset = Dataset.from_list(data)  # Convertir la liste en Dataset Hugging Face

    def translate_batch(batch):
        """Traduit un batch de textes."""
        translations = translator(batch["page_content"], truncation=True, max_length=512)
        return {"page_content": [translation["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks


# Enregistrer le document complet traduit
def save_translated_document(translated_chunks, output_file_path):
    """Enregistre tout le texte traduit dans un seul fichier .txt."""
    with open(output_file_path, "w", encoding="utf-8") as f:
        for chunk in translated_chunks["page_content"]:
            f.write(chunk + "\n")  # Ajouter un saut de ligne entre les morceaux
    print(f"Document traduit enregistré sous : {output_file_path}")


# Exemple d'utilisation
file_path = "/content/spark_tes.txt"  # Chemin du fichier source
output_file_path = "/content/translated_document.txt"  # Chemin pour le fichier traduit

data = load_data(file_path)

if data:
    translated_chunks = translate_with_pipeline(data, batch_size=16)
    save_translated_document(translated_chunks, output_file_path)
else:
    print("Aucune donnée à traduire.")

Device set to use cpu


Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Document traduit enregistré sous : /content/translated_document.txt


#Traduction de espagnol ---> français d'un dossier


Helsinki-NLP/opus-mt-es-fr

---



In [ ]:
!git lfs install
!git clone https://huggingface.co/Helsinki-NLP/opus-mt-es-fr

Git LFS initialized.
Cloning into 'opus-mt-es-fr'...
remote: Enumerating objects: 65, done.
remote: Total 65 (delta 0), reused 0 (delta 0), pack-reused 65 (from 1)
Unpacking objects: 100% (65/65), 1.40 MiB | 5.60 MiB/s, done.
Filtering content: 100% (2/2), 634.21 MiB | 76.40 MiB/s, done.


In [ ]:

import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset
from transformers import pipeline
import torch


# Charger le texte d'un fichier
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None

    if not texte:
        print(f"Le fichier {file_path} est vide.")
        return None

    # Diviser le texte en morceaux avec LangChain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Taille des morceaux
        chunk_overlap=0,  # Chevauchement entre morceaux
        length_function=len
    )

    # Créer les documents
    documents = text_splitter.create_documents([texte])

    # Extraire les textes bruts des documents pour compatibilité avec Dataset
    data = [{"page_content": doc.page_content} for doc in documents]

    return data


# Traduire les morceaux avec le pipeline
def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline("translation_es_to_fr", model="/content/opus-mt-es-fr", device=device)

    dataset = Dataset.from_list(data)  # Convertir la liste en Dataset Hugging Face

    def translate_batch(batch):
        """Traduit un batch de textes."""
        translations = translator(batch["page_content"], truncation=True, max_length=512)
        return {"page_content": [translation["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks


# Enregistrer le document traduit
def save_translated_document(translated_chunks, output_file_path):
    """Enregistre tout le texte traduit dans un seul fichier .txt."""
    with open(output_file_path, "w", encoding="utf-8") as f:
        for chunk in translated_chunks["page_content"]:
            f.write(chunk + "\n")  # Ajouter un saut de ligne entre les morceaux
    print(f"Document traduit enregistré sous : {output_file_path}")


# Traduire tous les fichiers dans un dossier
def translate_folder(input_folder, output_folder, batch_size=16):
    """Traduit tous les fichiers .txt d'un dossier et enregistre les traductions."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Créer le dossier de sortie s'il n'existe pas

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):  # Filtrer uniquement les fichiers .txt
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, f"{file_name}")

            print(f"Traitement du fichier : {input_file_path}")
            data = load_data(input_file_path)

            if data:
                translated_chunks = translate_with_pipeline(data, batch_size=batch_size)
                save_translated_document(translated_chunks, output_file_path)
            else:
                print(f"Fichier ignoré (vide ou erreur) : {file_name}")


# Exemple d'utilisation
input_folder = "/content/drive/MyDrive/test_espagnole"  # Dossier contenant les fichiers source
output_folder = "/content/translated_texts"  # Dossier pour les fichiers traduits

translate_folder(input_folder, output_folder, batch_size=16)

Traitement du fichier : /content/drive/MyDrive/test_espagnole/texte1.txt


Device set to use cuda:0


Map:   0%|          | 0/39 [00:00<?, ? examples/s]

Document traduit enregistré sous : /content/translated_texts/texte1.txt
Traitement du fichier : /content/drive/MyDrive/test_espagnole/texte2.txt


Device set to use cuda:0


Map:   0%|          | 0/37 [00:00<?, ? examples/s]

Document traduit enregistré sous : /content/translated_texts/texte2.txt


facebook/mbart-large-50-many-to-many-mmt

---



In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."
article_es = """ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
                Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
                Para ello ponemos a tu disposición:
                Información y herramientas para la prevención de riesgos y enfermedades, así como planes de cuidados para recuperar el bienestar en caso de enfermedad crónica o etapa especial de la vida.
                Programa La Salud Inteligente de Adeslas
                Dirigido a personas con diabetes, enfermedad cardiovascular (infarto o angina de pecho), con antecedentes o factores de riesgo (tabaquismo, glucosa o colesterol elevado, hipertensión arterial o sobrepeso).
                El 35% de la población en España respira aire contaminado
                Según el Instituto Nacional de Estadística (INE) y el Ministerio de Medio Ambiente, el 3% de la mortalidad anual en España se debe a la contaminación atmosférica.
                Identifícate en la cabecera de arriba con tu usuario y contraseña.
                Si no dispones de ellos, date de alta aquí
                Fumar afecta también a la piel
                Una razón más para dejar el hábito de fumar, los efectos del tabaco sobre la piel son numerosos y no solo estéticos.
                """

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer.src_lang = "es_ES"
encoded_ar = tokenizer(article_es, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['In order to do this, we make available to you: Information and tools for the prevention of risks and diseases, as well as care plans to recover the well-being in the case of chronic diseases and special stages of life. The Intelligent Health of Yourself Program The Intelligent Health of Yourself Program Directed to people with diabetes, cardiovascular disease (infarction or angina), with a history of risk factors (smoking, glucose, high cholesterol, hypertension or overweight). 35% of the population in Spain breathes contaminated air According to the National Institute of Statistics (INE) and the Ministry of the Environment, 3% of the annual death rate in Spain is due to air pollution. Identify in the top header with your user and password. If you do not have them, date up here Smoking affects also the skin Smoking affects also the skin Another reason to stop smoking, the effects of']

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch


# Initialize MBart model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load text from a file
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None

    if not texte:
        print(f"Le fichier {file_path} est vide.")
        return None

    # Split the text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=0,
        length_function=len,
    )

    documents = text_splitter.create_documents([texte])
    data = [{"page_content": doc.page_content} for doc in documents]
    return data

# Translate text chunks using MBart
def translate_with_mbart(data, src_lang="es_ES", tgt_lang="fr_XX", batch_size=16):
    dataset = Dataset.from_list(data)  # Convert list to Dataset

    def translate_batch(batch):
        """Translate a batch of text chunks."""
        inputs = tokenizer(
            batch["page_content"],
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True,
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        tokenizer.src_lang = src_lang

        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
            max_length=512,
        )
        translations = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return {"page_content": translations}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size,
    )
    return translated_chunks

# Save translated document
def save_translated_document(translated_chunks, output_file_path):
    with open(output_file_path, "w", encoding="utf-8") as f:
        for chunk in translated_chunks["page_content"]:
            f.write(chunk + "\n")
    print(f"Document traduit enregistré sous : {output_file_path}")

# Translate all files in a folder
def translate_folder(input_folder, output_folder, src_lang="es_ES", tgt_lang="fr_XX", batch_size=16):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, file_name)

            print(f"Traitement du fichier : {input_file_path}")
            data = load_data(input_file_path)

            if data:
                translated_chunks = translate_with_mbart(data, src_lang, tgt_lang, batch_size)
                save_translated_document(translated_chunks, output_file_path)
            else:
                print(f"Fichier ignoré (vide ou erreur) : {file_name}")

# Example usage
input_folder = "/content/drive/MyDrive/test_espagnole"
output_folder = "/content/translated_texts"

translate_folder(input_folder, output_folder, src_lang="es_ES", tgt_lang="fr_XX", batch_size=16)

Traitement du fichier : /content/drive/MyDrive/test_espagnole/texte1.txt


Map:   0%|          | 0/39 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 87.06 MiB is free. Process 117597 has 14.66 GiB memory in use. Of the allocated memory 14.36 GiB is allocated by PyTorch, and 183.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

facebook/nllb-200-distilled-600M

---



In [ ]:
translator = pipeline(
                        task="translation",
                        model="facebook/nllb-200-distilled-600M",
                        max_length=512,
                        device=device
                    )

In [ ]:

import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datasets import Dataset
from transformers import pipeline
import torch


# Charger le texte d'un fichier
def load_data(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            texte = f.read()
    except Exception as e:
        print(f"Erreur lors du chargement du fichier : {e}")
        return None

    if not texte:
        print(f"Le fichier {file_path} est vide.")
        return None

    # Diviser le texte en morceaux avec LangChain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,  # Taille des morceaux
        chunk_overlap=0,  # Chevauchement entre morceaux
        length_function=len
    )

    # Créer les documents
    documents = text_splitter.create_documents([texte])

    # Extraire les textes bruts des documents pour compatibilité avec Dataset
    data = [{"page_content": doc.page_content} for doc in documents]

    return data


# Traduire les morceaux avec le pipeline
def translate_with_pipeline(data, batch_size=16):
    """Traduit les textes par lots avec un pipeline."""
    device = 0 if torch.cuda.is_available() else -1
    translator = pipeline(
                                  task="translation",
                                  model="facebook/nllb-200-distilled-600M",
                                  max_length=512,
                                  device=device
                              )
    dataset = Dataset.from_list(data)  # Convertir la liste en Dataset Hugging Face

    def translate_batch(batch):
        """Traduit un batch de textes."""
        translations = translator(batch["page_content"], src_lang="spa", tgt_lang="eng")
        # translations = translator(batch["page_content"], truncation=True, max_length=512)
        return {"page_content": [translation["translation_text"] for translation in translations]}

    translated_chunks = dataset.map(
        translate_batch,
        batched=True,
        batch_size=batch_size
    )
    return translated_chunks


# Enregistrer le document traduit
def save_translated_document(translated_chunks, output_file_path):
    """Enregistre tout le texte traduit dans un seul fichier .txt."""
    with open(output_file_path, "w", encoding="utf-8") as f:
        for chunk in translated_chunks["page_content"]:
            f.write(chunk + "\n")  # Ajouter un saut de ligne entre les morceaux
    print(f"Document traduit enregistré sous : {output_file_path}")


# Traduire tous les fichiers dans un dossier
def translate_folder(input_folder, output_folder, batch_size=16):
    """Traduit tous les fichiers .txt d'un dossier et enregistre les traductions."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Créer le dossier de sortie s'il n'existe pas

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".txt"):  # Filtrer uniquement les fichiers .txt
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, f"{file_name}")

            print(f"Traitement du fichier : {input_file_path}")
            data = load_data(input_file_path)

            if data:
                translated_chunks = translate_with_pipeline(data, batch_size=batch_size)
                save_translated_document(translated_chunks, output_file_path)
            else:
                print(f"Fichier ignoré (vide ou erreur) : {file_name}")


# Exemple d'utilisation
input_folder = "/content/drive/MyDrive/test_espagnole"  # Dossier contenant les fichiers source
output_folder = "/content/translated_texts"  # Dossier pour les fichiers traduits

translate_folder(input_folder, output_folder, batch_size=16)

Traitement du fichier : /content/drive/MyDrive/test_espagnole/texte1.txt


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 313.06 MiB is free. Process 3149 has 14.44 GiB memory in use. Of the allocated memory 14.09 GiB is allocated by PyTorch, and 234.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!pip install "unbabel-comet>=2.2.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 15.8 MB/s eta 0:00:00


In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/XCOMET-XL")
model = load_from_checkpoint(model_path)
data = [
    {
        "src": "Boris Johnson teeters on edge of favour with Tory MPs",
        "mt": "Boris Johnson ist bei Tory-Abgeordneten völlig in der Gunst",
        "ref": "Boris Johnsons Beliebtheit bei Tory-MPs steht auf der Kippe"
    }
]
model_output = model.predict(data, batch_size=8, gpus=1)
# Segment-level scores
print (model_output.scores)

# System-level score
print (model_output.system_score)

# Score explanation (error spans)
print (model_output.metadata.error_spans)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

KeyError: "Model 'Unbabel/XCOMET-XL' not supported by COMET."

#spark dataset

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StringType, ArrayType, StructType, StructField
from transformers import pipeline
from datasets import Dataset
import torch

# Initialiser SparkSession
spark = SparkSession.builder \
    .appName("ParquetTranslationWithChunks") \
    .getOrCreate()

# Charger le fichier Parquet
df_paquet = spark.read.parquet("/content/drive/MyDrive/CoWeSe.parquet")

# Fonction pour diviser un texte en morceaux
def split_text(text, max_length=267):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        current_length += len(word) + 1  # Inclure l'espace
        if current_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# UDF pour appliquer le découpage de texte
split_text_udf = udf(lambda text: split_text(text), ArrayType(StringType()))

# Diviser le texte en morceaux
df_split = df_paquet.withColumn("chunks", split_text_udf(col("texte")))

# Exploser les morceaux en lignes individuelles
df_exploded = df_split.select(explode(col("chunks")).alias("chunk"))

# Transformer les données en format Dataset Hugging Face
device = 0 if torch.cuda.is_available() else -1
translator = pipeline(task="translation_sp_to_fr", model="t5-small", batch_size=16, device=device)

# Fonction pour traduire un lot de morceaux
def translate_batch(batch):
    dataset = Dataset.from_dict({"text": batch})
    translations = translator(dataset["text"], max_length=267, truncation=True)
    return [t["translation_text"] for t in translations]

# UDF pour traduire les morceaux
translate_udf = udf(lambda chunk: translate_batch([chunk])[0], StringType())

# Traduire les morceaux
df_translated = df_exploded.withColumn("translated_chunk", translate_udf(col("chunk")))

# Recombiner les morceaux en lignes d'origine
df_recombined = df_translated.groupBy().agg(
    # Combiner les morceaux traduits en une seule chaîne
    col("translated_chunk").alias("translated_text")
)

# Sauvegarder le résultat au format Parquet
df_recombined.write.parquet("/content/drive/MyDrive/CoWeSe_translated_final.parquet")

# Afficher les résultats pour vérification
df_recombined.show()

#Deepseek

In [ ]:
!git clone https://huggingface.co/deepseek-ai/DeepSeek-V3-Base

Cloning into 'DeepSeek-V3-Base'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 215 (delta 14), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (215/215), 2.43 MiB | 5.10 MiB/s, done.
Resolving deltas: 100% (14/14), done.
error: unable to write file model-00050-of-000163.safetensors
error: unable to write file model-00039-of-000163.safetensors
error: unable to write file model-00007-of-000163.safetensors
error: unable to write file model-00006-of-000163.safetensors
error: unable to write file model-00001-of-000163.safetensors
error: unable to write file model-00086-of-000163.safetensors
error: unable to write file model-00028-of-000163.safetensors
error: unable to write file model-00083-of-000163.safetensors
error: unable to write file model-00099-of-000163.safetensors
error: unable to write file model-00055-of-000163.safetensors
error: unable to write file mod

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1", trust_remote_code=True)
pipe(messages)

#facebook/mbart-large-50

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ro_RO")

src_text = " UN Chief Says There Is No Military Solution in Syria"
tgt_text =  "Şeful ONU declară că nu există o soluţie militară în Siria"

model_inputs = tokenizer(src_text, return_tensors="pt")

generated_tokens=model(**model_inputs)


In [ ]:
generated_tokens

In [ ]:
import torch

# Decode the translated output
predicted_ids = torch.argmax(generated_tokens.logits, dim=-1)  # Get predicted IDs
translated_text = tokenizer.decode(predicted_ids[0].tolist(), skip_special_tokens=True)

# Print the translated text
print("Translated text:", translated_text)


Translated text: Chief Says There Is No Military Solution in Syria


In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50")

# Set source and target languages (Spanish -> French)
src_lang = "es_XX"  # Spanish
tgt_lang = "fr_XX"  # French

# Text to translate (Spanish)
src_text = """ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN
                Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida.
                Para ello ponemos a tu disposición:"""

# Tokenize the source text with the appropriate source language token
tokenizer.src_lang = src_lang
model_inputs = tokenizer(src_text, return_tensors="pt")

# Generate translation
generated_tokens = model.generate(**model_inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

# Decode the translated output
translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

# Print the translated text
print("Translated text:", translated_text)


Translated text: ATENCIÓN: ESTA PÁGINA ESTÁ DESPROTEGIDA O EN MODO EDICIÓN Adeslas te ayuda a preservar y mejorar tu salud a lo largo de la vida. Para ello ponemos a tu disposición:
